In [3]:
import requests
import json
import time

import pandas as pd
from datetime import timedelta
from datetime import datetime
from config import gkey

In [4]:
#Input the target city and state
target_city = input("Enter city & state in the format 'city,state: ")

Enter city & state in the format 'city,state: Sayreville,NJ


In [5]:
#Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

In [6]:
#Creating the params dict with address and key
params = {"address": target_city, "key": gkey}

In [7]:
# Run request (Build URL using the base_url & params)
response = requests.get(base_url, params=params)
#print(response.url)

# Convert the response to JSON
city_geo = response.json()

# Extract Lat/Lng
lat = city_geo["results"][0]["geometry"]["location"]["lat"]
lng = city_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print("%s: %s, %s" % (target_city, lat, lng))

#Converting the lat & lng to string
lat = str(lat)
lng = str(lng)

Sayreville,NJ: 40.45940210000001, -74.360846


In [8]:
#https://developers.google.com/places/web-service/supported_types

#Segregated the places types into indoor and outdoor activities 
Acty_In = pd.read_excel("Resources/Indoor.xlsx")
Acty_Out = pd.read_excel("Resources/Outdoor.xlsx")

#Converting the dataframe into list
Acty_Outlist = Acty_Out["Outdoor"].tolist()
Acty_Inlist = Acty_In["Indoor"].tolist()

#Find the names and addresses of nearbyplaces in given city,state entered by the user.
#Hint: See https://developers.google.com/places/web-service/supported_types
Acty = input("Enter Outdoor or Indoor: ")

if (Acty == "Outdoor"):
    Acty = Acty_Outlist
elif (Acty == "Indoor"):
    Acty = Acty_Inlist

Enter Outdoor or Indoor: Indoor


In [9]:
#Google maps nearby places search API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

places_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"

In [ ]:
place_names = []
addresses = []
for i in range(len(Acty)):
    
    target_type = Acty[i]
    #radius = 40000

# Build URL using the Google Maps API and other fields
    #target_url = base_url+"key="+gkey+"&location="+lat+","+lng+"&radius="+str(radius)+"&type="+target_type#+"&opennow"
    target_url = base_url+"key="+gkey+"&location="+lat+","+lng+"&rankby=distance"+"&type="+target_type#+"&opennow"
    #print(target_url)
# Run request
    response = requests.get(target_url)
    city_placestogo = response.json()
    
    if city_placestogo["status"] == "ZERO_RESULTS":
        print(f'No {Acty[i]} near your place of choice')
        print("\n")
    else:
        print(f'Bingo!!! We have found good {Acty[i]}s near your place of choice!!!')
        print("\n")
        for item in city_placestogo["results"]:
            myplace = item["name"]
            myplace = myplace.replace(" ", "%20")
            target_url = places_url+"key="+gkey+"&input="+myplace+"&inputtype=textquery"+"&fields=formatted_address,name,rating,opening_hours"
            #print(target_url)
            response = requests.get(target_url)
            data = response.json()
            #print(data)
            try:
                print(f'{data["candidates"][0]["name"]}')
                print(f'{data["candidates"][0]["formatted_address"]}')
                print(f'The rating of {data["candidates"][0]["name"]} is {data["candidates"][0]["rating"]}')
                print(f'Is it open? {data["candidates"][0]["opening_hours"]["open_now"]}')
            except:
                print(f'{item["name"]}')
                print("No or incomplete information found")
        #print("\n")    

In [ ]:
places_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
for place in place_names: 
    myplace = place
    myplace = myplace.replace(" ", "%20")
    target_url = places_url+"key="+gkey+"&input="+myplace+"&inputtype=textquery"+"&fields=formatted_address,name,rating,opening_hours"
    #print(target_url)
    response = requests.get(target_url)
    data = response.json()
    #print(data)
    try:
        print(f'{data["candidates"][0]["name"]}')
        print(f'Address: {data["candidates"][0]["formatted_address"]}')
        print(f'Rating: {data["candidates"][0]["name"]} is {data["candidates"][0]["rating"]}')
        print(f'Is it open? {data["candidates"][0]["opening_hours"]["open_now"]}')
    except:
        print(f'{place}')
        print("No or incomplete information found")
    print("\n")    